In [1]:
!pip install opencv-python numpy


In [2]:
import cv2
import base64
import numpy as np
import time

from google.colab.output import eval_js
from google.colab.patches import cv2_imshow


In [9]:
!ls


sample_data


In [10]:
wget https://raw.githubusercontent.com/chuanqi305/MobileNet-SSD/master/deploy.prototxt -O MobileNetSSD_deploy.prototxt
wget https://raw.githubusercontent.com/chuanqi305/MobileNet-SSD/master/mobilenet_iter_73000.caffemodel -O MobileNetSSD_deploy.caffemodel


SyntaxError: invalid syntax (ipython-input-3084401467.py, line 1)

In [4]:
net = cv2.dnn.readNetFromCaffe(
    "MobileNetSSD_deploy.prototxt",
    "MobileNetSSD_deploy.caffemodel"
)


error: OpenCV(4.12.0) /io/opencv/modules/dnn/src/caffe/caffe_io.cpp:1126: error: (-2:Unspecified error) FAILED: fs.is_open(). Can't open "MobileNetSSD_deploy.prototxt" in function 'ReadProtoFromTextFile'


In [5]:
CLASSES = [
    "background", "aeroplane", "bicycle", "bird", "boat",
    "bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
    "dog", "horse", "motorbike", "person", "pottedplant",
    "sheep", "sofa", "train", "tvmonitor"
]

ALLOWED_CLASSES = ["person", "chair", "bottle", "tvmonitor"]
CONF_THRESHOLD = 0.6


In [6]:
def capture_frame():
    js = """
    async function capture() {
        const video = document.createElement('video');
        const stream = await navigator.mediaDevices.getUserMedia({video: true});
        video.srcObject = stream;
        await video.play();

        const canvas = document.createElement('canvas');
        canvas.width = video.videoWidth;
        canvas.height = video.videoHeight;
        const ctx = canvas.getContext('2d');
        ctx.drawImage(video, 0, 0);

        stream.getTracks().forEach(track => track.stop());
        return canvas.toDataURL('image/jpeg');
    }
    capture();
    """
    data = eval_js(js)
    img = base64.b64decode(data.split(',')[1])
    frame = cv2.imdecode(np.frombuffer(img, np.uint8), cv2.IMREAD_COLOR)
    return frame


In [ ]:
for i in range(10):  # near real-time frames

    frame = capture_frame()
    frame = cv2.resize(frame, (640, 480))
    (h, w) = frame.shape[:2]

    blob = cv2.dnn.blobFromImage(
        cv2.resize(frame, (300, 300)),
        0.007843,
        (300, 300),
        127.5
    )

    net.setInput(blob)
    detections = net.forward()

    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]

        if confidence < CONF_THRESHOLD:
            continue

        class_id = int(detections[0, 0, i, 1])
        label = CLASSES[class_id]

        if label not in ALLOWED_CLASSES:
            continue

        box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
        (x1, y1, x2, y2) = box.astype("int")

        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(
            frame,
            f"{label} {confidence:.2f}",
            (x1, y1 - 10),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.6,
            (0, 255, 0),
            2
        )

    cv2_imshow(frame)
    time.sleep(0.7)
